In [1]:
import pandas as pd
import requests
from multiprocessing.dummy import Pool
from multiprocessing import cpu_count, freeze_support
import shapely
from shapely import geometry
import re
import time
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
#логер
def log_progress(sequence, every=None, size=None):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display
    
    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = size / 200     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)
    
    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    label.value = u'{index} / {size}'.format(
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = str(index or '?')

In [29]:
#comments
def get_comment_element(url, driver, count_review):
    driver.get(url)
    time.sleep(5)
    try:
        item = driver.find_element_by_css_selector('button[jsaction="pane.reviewChart.moreReviews"]')
        driver.execute_script("arguments[0].click();", item)
        return get_all_comments_place(url, driver, count_review)
    except:
        return 
    
def scroll(driver, path, count):
    items = []
    flag = True
    try:
        item = driver.find_element_by_xpath("//div[@class = 'siAUzd-neVct section-scrollbox cYB2Ge-oHo7ed cYB2Ge-ti6hGc']")
        last_height = driver.execute_script("return arguments[0].scrollHeight", item)
    except:
        flag = False


    while flag:
#         try:
#             # Scroll down to bottom
#             driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight)", item)
#             new_height = driver.execute_script("return arguments[0].scrollHeight", item)
#         except:
#             new_height = last_height
#             pass
        new_height = last_height
        # Wait to load page
        start_time = time.time()
        while new_height == last_height:
            try:
                driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight)", item)
                item = driver.find_element_by_xpath("//div[@class = 'siAUzd-neVct section-scrollbox cYB2Ge-oHo7ed cYB2Ge-ti6hGc']")
                # Calculate new scroll height and compare with last scroll height
                new_height = driver.execute_script("return arguments[0].scrollHeight", item)          
                items = driver.find_elements_by_xpath(path)
                if (len(items) >= count) or (time.time()-start_time)>35:
                    flag = False
                    break
            except Exception as e:
                print('3 '+e)
                flag = False
                break
        last_height = new_height

    items = driver.find_elements_by_xpath(path) 
    return items
    
def get_all_comments_place(place_url, driver, count_review):
    
    time.sleep(6)
    
    reviews_obj_list = []

    reviews = scroll(driver, '//div[@class="ODSEW-ShBeI NIyLF-haAclf gm2-body-2"]', count_review)
    
    for review in log_progress(reviews, 1):

#         нажать еще
        try:
            btn = review.find_element_by_xpath("//jsl /button")
            driver.execute_script("arguments[0].click();", btn)
        except:
            pass
        
        id_review = ''
        try:    
            id_review = review.get_attribute('data-review-id')
        except:
            pass
        
        photos = []
        try:
            p = review.find_elements_by_class_name('ODSEW-ShBeI-xJzy8c')
            if p[-1].get_attribute('aria-label')[0]=='+':
                driver.execute_script("arguments[0].click();", p[-1])
                time.sleep(1)
                p = review.find_elements_by_class_name('ODSEW-ShBeI-xJzy8c')
            photos = [re.search(r'\"https://\w.*\"', i.get_attribute('style')).group(0).replace('"', '') for i in p]
        except:
            pass

        rating = ""
        try:
            rating = int(review.find_element_by_class_name("ODSEW-ShBeI-H1e3jb").get_attribute('aria-label')[1])
        except:
            pass

        pub_date_text = ""
        try:    
            pub_date_text = review.find_element_by_class_name("ODSEW-ShBeI-RgZmSc-date").text
        except:
            pass

        
        reviewer_level = ''
        reviewer_counts = 0
        reviewer_page = ""
        try:
            reviewer = review.find_element_by_class_name("ODSEW-ShBeI-tXxcle-SfQLQb-menu")
            reviewer_page = reviewer.find_element_by_tag_name('a').get_attribute('href')
            level_and_counts = reviewer.text.replace('\n', '·').split('·')[1:]
            reviewer_level = level_and_counts[0].strip()
            reviewer_counts = level_and_counts[1].strip().split()[0]
        except:
            pass

        text = ""
        try:
            text = review.find_element_by_class_name("ODSEW-ShBeI-text").text
        except:
            pass   
        
        count_supports = 0
        try:
            count_supports = int(review.find_element_by_class_name("ODSEW-ShBeI-hWJfub-yHKmmc-NnAfwf").text)
        except:
            pass 

        obj = {
                'url': place_url,
                'id_review': id_review,
                'reviewer_page': reviewer_page,
                'reviewer_level': reviewer_level,
                'reviewer_counts': reviewer_counts,
                'rating': rating,
                'likes':count_supports,
                'date': pub_date_text,
                'text':text,
                'photos': photos
        }

        reviews_obj_list.append(obj)
    
    return reviews_obj_list

In [5]:
places_df = pd.read_csv('../data/places_GP.csv')
comments_df = pd.read_csv('../data/comments.csv')

In [22]:
temp_df = comments_df.groupby('url').count()['text'].reset_index()
comments_df = comments_df.loc[comments_df.url.isin(temp_df.loc[temp_df.text>8, 'url'])]
places_df.loc[(~places_df.count_review.isna()) & ~places_df.url.isin(comments_df.url)]

,url,title,address,type,popular_times_bars,geometry,types,rating,count_review,count_photo,category_2,category,normalized_rating,rating_by_category
0,https://www.google.com/maps/place/?q=place_id:...,"Новый век, СФ","ул. Демуса, 60, Краснодар, Краснодарский край,...",Строительная компания,NaN,POINT (39.09852739999999 45.0494082),"['general_contractor', 'point_of_interest', 'e...",5.0,2.0,3.0,NaN,NaN,1.000106,0.000000
1,https://www.google.com/maps/place/?q=place_id:...,Магазин автоприцепов,"ул. Ростовское ш., 24/1, Краснодар, Краснодарс...",Продажа прицепов,NaN,POINT (38.9911055 45.08962769999999),"['point_of_interest', 'store', 'establishment']",4.5,2.0,6.0,Магазин,Магазин,0.900156,0.900347
6,https://www.google.com/maps/place/?q=place_id:...,Олимпия,"ул. Крупской, 22, Краснодар, Краснодарский кра...",NaN,NaN,POINT (39.1108766 45.0162226),"['lodging', 'point_of_interest', 'establishment']",4.1,103.0,56.0,Жилье,Услуга,0.823806,0.901502
8,https://www.google.com/maps/place/?q=place_id:...,Репетиторский Центр,"ул. Измаильская, 74 к 4, Краснодар, Краснодарс...",Образовательный центр,NaN,POINT (39.0456162 45.11374360000001),"['point_of_interest', 'establishment']",5.0,4.0,14.0,NaN,NaN,1.000344,0.000000
9,https://www.google.com/maps/place/?q=place_id:...,"Вэлданс, студия танца и йоги","ул. 70-летия Октября, 12/3, Краснодар, Краснод...",Школа танцев,NaN,POINT (38.9117982 45.0379148),"['point_of_interest', 'establishment']",5.0,1.0,NaN,NaN,NaN,1.000028,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15520,https://www.google.com/maps/place/?q=place_id:...,Мадьяр-Гриль,"ул. Дзержинского, 100, Краснодар, Краснодарски...",Ресторан,"['Загруженность в 06:00: 0%.', 'Загруженность ...",POINT (38.9831636 45.1007007),"['restaurant', 'bar', 'food', 'point_of_intere...",4.6,170.0,32.0,Общепит,Общепит,0.925277,0.976035
15521,https://www.google.com/maps/place/?q=place_id:...,DNS,"ул. Дзержинского, 100, Краснодар, Краснодарски...",Магазин электроники,"['Загруженность в 06:00: 0%.', 'Загруженность ...",POINT (38.983847 45.10220899999999),"['electronics_store', 'home_goods_store', 'poi...",4.1,71.0,19.0,Магазин,Магазин,0.822298,0.823450
15522,https://www.google.com/maps/place/?q=place_id:...,"Ригла, ООО, сеть аптек","ул. Кубанская набережная, 25А, Краснодар, Крас...",Аптека,"['Загруженность в 04:00: 0%.', 'Загруженность ...",POINT (38.9594357 45.0219355),"['pharmacy', 'health', 'point_of_interest', 's...",2.3,9.0,4.0,Магазин,Магазин,0.460318,0.460510
15523,https://www.google.com/maps/place/?q=place_id:...,"Патрик & Мари, Сеть кулинарий","ул. Кубанская набережная, 35, Краснодар, Красн...",Кафе-кондитерская,"['Загруженность в 04:00: 0%.', 'Загруженность ...",POINT (38.9594648 45.0246223),"['meal_delivery', 'bakery', 'food', 'point_of_...",4.5,225.0,188.0,Общепит,Общепит,0.909406,0.993235


In [23]:
d = {0:[],1:[],2:[],3:[],4:[],5:[],6:[]}
for i, r in places_df.loc[(~places_df.count_review.isna()) & ~places_df.url.isin(comments_df.url)].sort_values('count_review', ascending = False, ignore_index=True).iterrows():
    d[i%7].append((r['url'], r['count_review']))

In [24]:
def get_reviews(urls):
    driver = webdriver.Chrome('chromedriver.exe')
    for url, count in log_progress(urls, 1):
        time.sleep(3)
        try:
            comments.extend(get_comment_element(url, driver, count))
        except:
            pass

In [26]:
comments = []

if __name__== "__main__":
    freeze_support()
    pool = Pool()
    ITERATION_COUNT = cpu_count()-1
    for i in range(0, ITERATION_COUNT):
        pool.apply_async(get_reviews, [d[i]])

In [38]:
c_df = pd.DataFrame(comments)

In [37]:
pd.concat([comments_df, c_df])

,url,id_review,reviewer_page,reviewer_level,reviewer_counts,rating,likes,date,text,photos
0,https://www.google.com/maps/place/?q=place_id:...,NaN,https://www.google.com/maps/contrib/1063546855...,Местный эксперт,29,5.0,0,неделю назад,"Есть доставка.\nБольшой ассортимент продуктов,...",[]
1,https://www.google.com/maps/place/?q=place_id:...,NaN,https://www.google.com/maps/contrib/1088620985...,Местный эксперт,25,5.0,0,3 недели назад,"В Гипер Ленте есть почти всë и преимущество, ч...",[]
2,https://www.google.com/maps/place/?q=place_id:...,NaN,https://www.google.com/maps/contrib/1150924961...,Местный эксперт,38,4.0,1,2 месяца назад,Хороший выбор товаров на любой вкус и кошелёк....,[]
3,https://www.google.com/maps/place/?q=place_id:...,NaN,https://www.google.com/maps/contrib/1102283153...,16 отзывов,0,5.0,0,неделю назад,"Хороший гипермаркет,с приятными ценами и скидк...",[]
4,https://www.google.com/maps/place/?q=place_id:...,NaN,https://www.google.com/maps/contrib/1054766260...,Местный эксперт,11,5.0,0,6 дней назад,Обожаю Ленту за огромный выбор товаров на все ...,[]
...,...,...,...,...,...,...,...,...,...,...
43706,https://www.google.com/maps/place/?q=place_id:...,ChZDSUhNMG9nS0VJQ0FnSURRNXFfb0FnEAE,https://www.google.com/maps/contrib/1111911437...,Местный эксперт,59,5,0,3 года назад,,[]
43707,https://www.google.com/maps/place/?q=place_id:...,ChZDSUhNMG9nS0VJQ0FnSURnX01IdEFREAE,https://www.google.com/maps/contrib/1135669361...,,0,4,0,3 года назад,,[]
43708,https://www.google.com/maps/place/?q=place_id:...,ChZDSUhNMG9nS0VJQ0FnSURndDc3bEFREAE,https://www.google.com/maps/contrib/1023406551...,Местный эксперт,0,5,0,3 года назад,,[]
43709,https://www.google.com/maps/place/?q=place_id:...,ChRDSUhNMG9nS0VJQ0FnSUNDM0xsNhAB,https://www.google.com/maps/contrib/1053626629...,,0,5,0,год назад,,[]


In [35]:
pd.concat([comments_df, c_df]).to_csv('../data/comments.csv', index = False)

In [221]:
comments_df

,url,id_review,reviewer_page,reviewer_level,reviewer_counts,rating,likes,date,text,photos
0,https://www.google.com/maps/place/?q=place_id:...,,https://www.google.com/maps/contrib/1063546855...,Местный эксперт,29,5,0,неделю назад,"Есть доставка.\nБольшой ассортимент продуктов,...",[]
1,https://www.google.com/maps/place/?q=place_id:...,,https://www.google.com/maps/contrib/1088620985...,Местный эксперт,25,5,0,3 недели назад,"В Гипер Ленте есть почти всë и преимущество, ч...",[]
2,https://www.google.com/maps/place/?q=place_id:...,,https://www.google.com/maps/contrib/1150924961...,Местный эксперт,38,4,1,2 месяца назад,Хороший выбор товаров на любой вкус и кошелёк....,[]
3,https://www.google.com/maps/place/?q=place_id:...,,https://www.google.com/maps/contrib/1102283153...,16 отзывов,0,5,0,неделю назад,"Хороший гипермаркет,с приятными ценами и скидк...",[]
4,https://www.google.com/maps/place/?q=place_id:...,,https://www.google.com/maps/contrib/1054766260...,Местный эксперт,11,5,0,6 дней назад,Обожаю Ленту за огромный выбор товаров на все ...,[]
...,...,...,...,...,...,...,...,...,...,...
63017,https://www.google.com/maps/place/?q=place_id:...,,https://www.google.com/maps/contrib/1031220674...,3 отзыва,0,5,0,2 года назад,Стейк из телятины средней прожарки великолепный!,[https://lh5.googleusercontent.com/p/AF1QipPLz...
63018,https://www.google.com/maps/place/?q=place_id:...,,,,0,4,0,3 года назад,,[https://lh5.googleusercontent.com/p/AF1QipOc8...
63019,https://www.google.com/maps/place/?q=place_id:...,,,,0,,0,,,[]
63020,https://www.google.com/maps/place/?q=place_id:...,,,,0,,0,,,[]
